## 함수

파이썬에서 프로그래머가 처음으로 사용하는 구성도구가 함수이다.
함수를 사용하면 가독성이 높아지고 재사용이나 리팩토링이 가능해진다.  

파이썬에서 제공하는 함수들에는 개발자를 좀 더 편하게 해주는 다양한 기능들이 있는데, 그 기능들을 소개한다.  
일부는 다른 프로그래밍 언어에도 존재하고, 다른 일부는 파이썬에만 있는 기능이다.  


1. None을 반환하기 보다는 예외를 일으키자
2. 클로저와 스코프
3. 리스트 대신 제너레이터를 고려하자, 단 인수를 순회할 때는 방어적으로 하자(이터레이터 프로토콜)
4. 가변위치 인수, 키워드위치 인수, 동적 기본인수의 지정


이러한 부가기능을 통해 __불필요한 요소를 제거하고 호출자의 의도를 명료하게 보여주며, 찾기 어려운 미묘한 버그를 상당수 줄여줄 수 있다.__

## 1. None을 반환하기보다는 예외를 일으키자

None 값을 사용하는 것은 파이썬 프로그래밍에 있어 오류를 발생시킬 가능성이 크다. 그렇기 때문에 최대한 예외처리를 사용하는 것이 바람직하다.


In [1]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None
x = 0
y = 10
result = divide(x, y)
if result is None:
    print('Invalid inputs')

In [2]:
x = 10
y = 0
result = divide(x, y)
if result is None:
    print('Invalid inputs')

Invalid inputs


## 2. 클로저와 스코프

클로저란. 자신이 정의된 스코프에 있는 변수를 참조하는 함수다.  
바로 이 점 덕분에 helper 함수가 sort_priority의 group 인수에 접근 할 수 있다.

- 함수는 파이썬에서 일급 객체다. 이 말은 함수를 직접 참조하고, 변수에 할당하고, 다른 함수의 인수로 전달하고, 표현식과 if else 문 등에서 비교할 수 있다는 말이다.
- 파이썬에서는 튜플을 비교하는 특정한 규칙이 있다. 먼저 인덱스를 0으로 아이템을 비교하고, 인덱스 1, 2 순으로 비교한다. helper 클로저의 반환값이 정렬 순서를 분리된 두 그룹으로 나뉘게 한 것이 이 규칙 때문이다.

In [3]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return(0, x)
        return (1, x)
    
    values.sort(key = helper)

In [10]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}

sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


In [15]:
#함수의 스코프를 잘못 설정한 경우
def sort_priority2(values, group):
    found = False
    def helper(x):
        if x in group:
            found = True
            return(0, x)
        return (1, x)
    
    values.sort(key = helper)
    return found

In [16]:
found = sort_priority2(numbers, group)
print('Found : ', found)
print(numbers)

Found :  False
[2, 3, 5, 7, 1, 4, 6, 8]


### 파이썬의 스코프 탐색 순서
1. 현재 함수의 스코프
2. (현재 스코프를 담고 있는 함수와 같이) 감싸고 있는 스코프
3. 코드를 포함하고 있는 모듈의 스코프(전역 스코프라고도 함)
4. (len이나 str 같은 함수를 담고 있는) 내장 스코프

변수가 이미 현재 스코프에 정의되어 있다면 새로운 값으로 인식하고 다르게 취급한다.  
파이썬은 변수가 현재 스코프레 존재하지 않으면 변수 정의로 취급한다.

이를 조절하기 위해 나온 것이 __\*nonlocal\*과 global__ 문 이다  

### nonlocal과 global

앞서 말한 것과 같은 문제를 해결하기 위해, 변수의 스코프를 개발자가 직접 정할 수 있게 하는 역할을 한다.

global 문은 변수 할당이 모듈 스코프에 직접들어 가게하여 전역변수를 생성한다.  
이에 반해 __nonlocal__ 문은 변수 할당이 현재보다 상위 스코프 까지 이루어진다.  


In [17]:
#nonlocal함수를 통해 변수의 스코프를 알맞게 설정한 경우
def sort_priority3(values, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return(0, x)
        return (1, x)
    
    values.sort(key = helper)
    return found

In [18]:
found = sort_priority3(numbers, group)
print('Found : ', found)
print(numbers)

Found :  True
[2, 3, 5, 7, 1, 4, 6, 8]


## 3. 리스트를 순회하는대신 제너레이터를 고려하자

제너레이터의 특징은 다음과 같다.
- 제너레이터를 사용하는 방법이 누적된 결과의 리스트를 반환하는 방법보다 이해하기 명확하다
- 제너레이터에서 반환한 이터레이터는 제너레이터 함수의 본문에 있는 __yield 표현식에 전달된 값들의 집합__ 이다.
- 제너레이터의 __모든 입력과 출력을 메모리에 저장하지 않으므로__ 입력값의 양을 알기 어려울 때도 연속된 출력을 만들 수 있다.

In [19]:
#리스트를 순회하는 방식의 함수 예제
def index_words(text):
    result=[]
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter==' ':
            result.append(index + 1)
    return result


In [21]:
address = 'Four score and seven years age...'
result = index_words(address)
print(result)

[0, 5, 11, 15, 21, 27]


In [22]:
#제너레이터를 사용하여 작성한 같은 함수
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter==' ':
            yield index + 1

In [24]:
address = 'Four score and seven years age...'
result = list(index_words_iter(address))
print(result)

[0, 5, 11, 15, 21, 27]


제너레이터를 안정적으로 구현하기 위해서는 방어적으로 코드를 짜는 과정이 필요하다.  

이는 제너레이터가 한번 출력한 값을 다시 불러올 수 없다는 특징 때문인데, 이를 위해서는 입력 __이터레이터를 방어적으로 복사해놓는 과정이 필요__ 하다.  

예를 들어 아래는 미국 텍사스주의 여행자 수를 분석하고 싶고, 각각의 퍼센트를 알고 싶다고 하자.

In [25]:
#일반적인 version
def normalize(numbers):
    total = sum(numbers)
    result=[]
    for value in numbers:
        percent = 100*value/total
        result.append(percent)
    return result

In [27]:
#리스트 전체를 넣어 계산하는 경우에는 잘 작동하는 것을 확인할 수 있다.
visits=[15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [30]:
#제너레이터를 잘못 활용한 경우(방어적 x), 이를 제너레이터로 불러온 함수와 같이 사용할 시 에러가 발생함을 알 수 있다.
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

In [29]:
it = read_visits('./dataset/my_numbers.txt')
percentages = normalize(it)
print(percentages)

[]


In [31]:
#제너레이터를 다루기 위한 normalize 함수
def normalize_copy(numbers):
    numbers = list(numbers) #이터레이터를 복사함
    total = sum(numbers)
    result=[]
    for value in numbers:
        percent = 100*value/total
        result.append(percent)
    return result

In [33]:
#이제 함수가 잘 작동함을 알 수 있다
it = read_visits('./dataset/my_numbers.txt')
percentages = normalize_copy(it)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


위와 같은 문제를 피하는 방법은
1. 호출될 때 마다 새 이터레이터를 반환하는 함수를 받게 만든다.
2. __이터레이터 프로토콜을 구현한 새 컨테이너 클래스를 제공하는 것이다.__
- 이 때는 제대로 작동하는 것을 볼 수 있는데, 이는 sum을 계산할 때 Readvisits.\__iter__를 호출하기 때문이다.  
- 이는 입력데이터를 매번 불러온다는 단점을 가지고는 있지만 메모리적으로 상당히 효율적인 방법이라고 볼 수 있다.

In [36]:
class Readvisits(object):
    def __init__(self, data_path):
        self.data_path = data_path
    
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

In [37]:
visits = Readvisits('./dataset/my_numbers.txt')
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


## 4. 가변위치인수, 동적인수

### 4-1. 가변위치인수
- 선택적인 위치인수인자를 관례적으로 \*args(star args)라고 한다. 이를 통해 함수 호출을 보다 명확하게 할 수 있다.  
- \*를 원하는 인수 앞에 지정함으로써 가변인수를 사용할 수 있다.
- 제너레이터와 \*연산자를 함께 사용하면 메모리 부족으로 프로그램이 망가질 수도 있다.


In [40]:
def log(message, *values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print('%s: %s' %(message, values_str))

In [41]:
log('My numbers are', 1, 2, 3 )
log('Hi there')

My numbers are: 1, 2, 3
Hi there


### 4-2. 동적인수
키워드 인수의 기본값으로 비정적 타입을 사용해야할 때도 있다. 예를 들어 이번트 발생 시각까지 포함해 로깅 메세지를 출력한다고 하자

In [52]:
from datetime import datetime
from time import sleep

def log(message, when = datetime.now()):
    print('%s: %s' %(when, message))

In [53]:
log('Hi there!')
sleep(0.1)
log('Hi there!')

2020-01-10 10:55:16.427331: Hi there!
2020-01-10 10:55:16.427331: Hi there!


이 때 시간이 바뀌지 않는 이유는 처음 log 함수가 실행 될 때 datetime.now()가 실행되고, 고정된 값으로 받아들이 때문이다.  
이를 해결하기 위해서는 __None으로 인자__ 를 받아야한다.

In [61]:
def log(message, when = None):
    when = datetime.now() if when is None else when
    print('%s: %s' %(when, message))

In [62]:
log('Hi there!')
sleep(0.1)
log('Hi there!')

2020-01-10 10:57:39.071572: Hi there!
2020-01-10 10:57:39.172584: Hi there!


추가적으로 위치인자와 키워드 인자를 정의하는 방법은 다음과같다.
- \*를 기준으로  앞이 위치인자, 뒤가 키워드 인자 순이고, 순서가 맞다면 \* 인자는 제거해도 에러 없이 돌아간다.  
`
def safe_division(number, divisor, *, ignore_overflow=True, ignore_zero_division=False)
`
